
# Summary

Today, we're going to be implementing least squares regression subject to a bound on *total variation*. That is, we're going to find a function $\hat \mu:\mathbb{R} \to \mathbb{R}$ that solves this optimization problem.

$$
\begin{aligned}
\hat \mu &= \operatorname*{argmin}_{\substack{m \\ \rho_{TV}(m) \le B}} \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 \qquad \text{ where } \\  & \rho_{TV}(m) = \max_{\substack{\text{finite sequences} \\ x_1 \le x_2 \le \ldots \le x_k \in \R }}
                  \sum_{j=1}^{k-1} \left\lvert m(x_{j+1})-m(x_j) \right\rvert.
\end{aligned}
$$

This is going to be a lot like implementing monotone regression. But there is another issue that we didn't have to deal with before. We have to choose a sensible value of the variation budget $B$. We'll take a look at how this impacts the curve we get and consider *cross-validation* as a strategy for choosing it automatically.


## Code we're Borrowing 
We'll use a few libraries.

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
})

# OSQP claims some feasible problems aren't, so we'll tell CVXR not to use it
CVXR::add_to_solver_blacklist('OSQP')  

# And we'll style our plots  
theme_update(plot.background = element_rect(fill = "transparent", colour = NA),
		    panel.background = element_rect(fill = "transparent", colour = NA),
                    legend.background = element_rect(fill="transparent", colour = NA),
                    legend.box.background = element_rect(fill="transparent", colour = NA),
                    legend.key = element_rect(fill="transparent", colour = NA),
			panel.grid.major=element_line(color=rgb(1,0,0,.1,  maxColorValue=1)),
	        panel.grid.minor=element_line(color=rgb(0,0,1,.1,  maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))

And we'll use the function `invert.unique` from the monotone regression lab. We'll be using this all the time. It takes a vector $X$ and returns a list of two things.

1. A vector `elements` cointaining the positions of the unique elements of $X$, so that `X[elements]` is vector of unique elements of $X$ sorted in increasing order. 
    - This vector has length $p$ where $p$ is the number of unique elements in $X$.
2. A vector `inverse` that tells you, for each $i$, the position of $X[i]$ in $X[elements]$. 
    - This vector has length $n$ where $n$ is the length of $X$.
    - `X[elements][inverse]` is the same as `X`.

In [ ]:
invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

a = c(1,2,3,3,4,5,5,5,6)
unique.a=invert.unique(a)
stopifnot(a[unique.a$elements][unique.a$inverse] == a)

This function, too, which is useful for turning a 'model object' into a function $\hat\mu$ that we can use to make predictions.

In [ ]:
prediction.function = function(model) {
  function(x) { predict(model, data.frame(X=x)) }
}

# Implementation

Much like we did with monotone regression, we'll use restriction and expansion. Here are the steps.

1. We'll start by solving for a function $\hat\mu_{\mid \mathcal{X}}:\mathcal{X} \to \R$ of the sample $\mathcal{X}=X_1 \ldots X_n$.
2. We'll extend $\hat\mu_{\mid \mathcal{X}}$ we get to a function $\hat\mu:\R \to \R$ on the real line that solves the original problem. 

## Step 1. Solving for $\hat\mu_{\mid \mathcal{X}}$
To do the first step, we'll have to think about what bounded variation means for a function from $\mathcal{X}$ to $\R$.  Recall from class 
the notion of bounded variation we've been using for functions $m:\R \to \R$. 
$$
\rho_{TV}(m) = \sup_{\substack{\text{finite sequences} \\ x_1 \le x_2 \le \ldots \le x_k \in \R}}
                  \sum_{j=1}^{k-1} \left\lvert m(x_{j+1})-m(x_j) \right\rvert.
$$
Here we're maximizing over all finite increasing sequences $x_1 \le x_2 \le \ldots \le x_k$ with points $x_j$ on the real line $\R$.
If what we're solving for is a function $\hat\mu_{\mid \mathcal{X}}$ on the sample instead of a function on the real line, it's natural to maximize over all finite increasing sequences $x_1 \le x_2 \le \ldots \le x_k$ with points $x_j$ in in the sample $\mathcal{X}$. 
$$
\rho_{TV\mid \mathcal{X}}(m) = \max_{\substack{\text{finite sequences} \\ x_1 \le x_2 \le \ldots \le x_k \in \mathcal{X}}}
                  \sum_{j=1}^{k-1} \left\lvert m(x_{j+1})-m(x_j) \right\rvert.
$$
And because the triangle inequality $\lvert x_{j+1}-x_j \rvert \le \lvert x_{j+1}-x^\star \rvert + \lvert x^\star - x_j \rvert$ tells us that adding a point $x^\star$ to a sequence $x_1 \ldots x_k$ can only increase the variation $\sum_{j=1}^{k-1} \lvert m(x_{j+1})-m(x_j) \rvert$, we know that using the whole sample, i.e. using the sequence $x_1 \ldots x_k = X_1 \ldots X_n$ assuming that the sample has been sorted in increasing order, will give us the maximal variation. This gives us a nice simple expression for $\hat\rho_{TV}(m)$ with no maximization involved.
$$
\rho_{TV \mid \mathcal{X}}(m) = \sum_{j=1}^{n-1} \left\lvert m(X_{j+1})-m(X_j) \right\rvert \qquad \text{when} \qquad X_1 \le \ldots \le X_n 
$$

This means we can solve for $\hat\mu_{\mid \mathcal{X}}$ by solving the following optimization problem.
$$
\begin{aligned}
\hat \mu_{\mid \mathcal{X}} &= \operatorname*{argmin}_{\substack{m:\mathcal{X} \to \R \\ \rho_{TV \mid \mathcal{X}}(m) \le B}} 
            \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 
\end{aligned}
$$ 

You can solve for $\hat\mu_{\mid \mathcal{X}}(X_1) \ldots \hat\mu_{\mid \mathcal{X}}(X_n)$ using CVXR code a lot like the code we used for monotone regression.

To save you some trouble, I've copied my monotone regression code into the block below. And I've handled some boilerplate changes for you, too.

1.  I've changed the name and arguments of the function (line 1).
2.  I've changed the 'class' attribute in the output so *R* knows it's a bounded variation regression rather than a monotone one (line 36).

That means that all you have to do is change the constraint. The one that's on line 22 in the cell below. 

In [ ]:
bvreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list( diff(m) >= 0 )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.bvreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "bvreg"
  model
}

## Step 2. Extending $\hat\mu_{\mid \mathcal{X}}$ to the real line 

We have code from the monotone regression lab for extending a function from the sample to the real line. Piecewise-constant extension code.

Here it is. I've changed the name to `predict.bvreg` so `predict(...)` knows to use it when we're using a `bvreg` model. 

In [ ]:
# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.piecewise.constant = function(model, newdata=data.frame(X=model$input$X)) {
  y = model$mu.hat; X=model$X; x=newdata$X
  # for each new data point x in newdata$X, 
  # find the closest observed X[k] left of x
  # i.e., the largest k for which X[k] <= x 
  # this covers cases 1 and 2
  # i will be a vector of these numbers k, with one for each x in newdata$X
  i = findInterval(x, X) 
  # if there is no X[k] < x, findInterval tells us k=0
  # to cover case 3, we want X[k] for k=1 when this happens.
  i[i==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  y[i]
}
predict.bvreg = predict.piecewise.constant

Let's take a look at what it does when we fit the same simple 'stepline' function we used in the monotone regression lab.

In [ ]:
mu = function(x) { x*(x >= .5) }
sigma = .1

n = 10
X = seq(0,1,length.out=n)
Y = mu(X) + sigma*rnorm(n)


model   = bvreg(X,Y, B=1)
muhat = prediction.function(model)

x = seq(0,1,by=.001) 
fit.on.sample = ggplot() + 
	geom_point(aes(x=X,y=Y),  alpha=.2) + 
	geom_line(aes(x=x,y=mu(x)), alpha=1) +
	geom_point(aes(x=X,y=mu(X)), color='blue', alpha=.2) +
	geom_line(aes(x=x,y=muhat(x)), color='blue')
	
fit.on.sample

It looks like it's doing a pretty good job fitting the data.  If you want, play around with it. Try changing the variation budget $B$ and look at how the fit changes. Or try changing the function $\mu$ from the stepline to something else and see how the fit changes.

## Is it the right extension?

Is this piecewise-constant extension really the solution to our original bounded variation regression problem? This one. Let's do an exercise.

$$
\begin{aligned}
\hat \mu &= \operatorname*{argmin}_{\substack{m \\ \rho_{TV}(m) \le B}} \ell(m) \quad \text{for} \quad \ell(m) =\frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 \qquad \text{ where }  \rho_{TV}(m) = \max_{\substack{\text{increasing sequences} \\ x_1 \le x_2 \le \ldots \le x_k}}
                  \sum_{j=1}^{k-1} \left\lvert m(x_{j+1})-m(x_j) \right\rvert.
\end{aligned}
$$


::: {.callout-exercise}
Suppose $\hat\mu_{\mid \mathcal{X}}: \mathcal{X} \to \R$ is a solution to our restricted least squares problem, i.e., it minimizes $\ell(m)$ over the 
set of functions $\{ m:\mathcal{X} \to \R \ : \ \rho_{TV \mid \mathcal{X}}(m) \le B \}$.  

 - Does the piecewise-constant extension of $\hat\mu_{\mid \mathcal{X}}$ solve the original bounded variation regression problem? 
 - If so, prove it.
 - If not, propose an alternative; implement it as `predict.bvreg`, and prove that extension solves the original problem.
:::


How do we think about this?  If we want to show that a function $\hat\mu$ solves a constrained optimization problem, we need to show two things.

1.  $\hat\mu$ is feasible, i.e. it satisfies the constraints. In this case, $\rho_{TV}(\hat\mu) \le B$.
2.  $\hat\mu$ is optimal, i.e., the value of the loss function we're minimizing, in this case mean squared error $\ell(m)$, is at least as small at $\hat\mu$ as for any other feasible function. In this case, any other function $m$ with $\rho_{TV}(m) \le B$.

Usually we do the optimality step by contradiction, i.e., we observe that if there were some function $m:\R \to \R$ with $\rho_{TV}(m) \le B$ and $\ell(m) < \ell(\hat\mu)$, then something we know to be true about $\hat\mu$ couldn't be true. In particular, when $\hat\mu$ is an extension of a function $\hat\mu_{\mid\mathcal{X}}$ that solves an optimization problem over functions from $\mathcal{X} \to \R$, we tend to do this by showing that if there were such a function $m$, then the restriction $m_{\mathcal{X}}$ would beat $\hat\mu_{\mid \mathcal{X}}$ at the game we know that $\hat\mu_{\mid \mathcal{X}}$ wins.


# Behavior and Tuning

Now that we've got some bounded variation regression code, let's see how it works and how that depends on the variation budget $B$. We'll try fitting data sampled around four simple curves. I've used the variation budget $B=1$ for all of them.

In [ ]:
n=800
sigma = .5
mus = list(step     = function(x) { 1*(x >= .5) },
	         line     = function(x) { x },
	         stepline = function(x) { x*(x >= .5) }, 
	         sin      = function(x) { sin(pi*x) })
           
x = seq(0,1,by=.01)
for(mu.name in names(mus)) { 
  mu = mus[[mu.name]]
  X     = runif(n)
  epsilon     = sigma*rnorm(length(X))
  Y = mu(X) + epsilon

  model   = bvreg(X,Y, B=2)
  muhat = prediction.function(model)

  plt = ggplot() + 
        geom_line(aes(x=x, y=mu(x)), alpha=.5) + 
        geom_point(aes(x=X, y=Y), alpha=.2)  + 
        geom_line(aes(x=x, y=muhat(x)), color='blue')
  print(plt)
}



For each curve $\mu$, we'll do the following.

1.  Sample $n=200$ some observations $(X_i,Y_i)$ with

    -   $X_i$ uniformly distributed on $[0,1]$
    -   $Y_i = \mu(X_i) + \varepsilon_i$ where $\varepsilon_i \sim N(0,\sigma^2)$ for $\sigma=.5$.

    And sample independent copies $(\tilde X_i,\tilde Y_i)$ from the same distribution.

2.  Use our code to find estimates $\hat\mu$ of the curve using variation budgets $B \in \{1/10,\ 1/4,\ 1/2,\ 1,\ 2,\ 4,\ 10\}$.

3.  For each of these estimates, calculate

    -   Training Error, $\frac1n\sum_{i=1}^n \{ \hat\mu(X_i) - Y_i \}^2$
    -   Test Error, $\frac1n\sum_{i=1}^n \{ \hat\mu(\tilde X_i) - \tilde Y_i \}^2$
    -   Population MSE, $\int_0^1 \{ \hat\mu(x) - \mu(x)\}^2 dx + \sigma^2$.  

    Adding $\sigma^2$ to the last error makes it comparable to the others, which are sensitive to the noise level.

We'll do it all 10 times and average the results to get a decent approximation to the expectation of each of these error measures. Let's run the blocks below to tabulate there errors. It will take a few minutes.

In [ ]:
training.error = function(mu.hat, d) { 
  mean((mu.hat(d$X)-d$Y)^2)
}
test.error     = function(mu.hat, d) { 
  mean((mu.hat(d$Xtest)-d$Ytest)^2)
}
population.mse = function(mu.hat, d) { 
  x=seq(0,1,by=.001)
  mean((mu.hat(x) - d$mu(x))^2) + sigma^2
}   
errors = list(training=training.error, 
              test=test.error, 
              population=population.mse)

Bs = c(.1,.25,.5,1,2,4,10)
models = Bs |> 
  map(\(B) \(X,Y) bvreg(X,Y,B)) |> 
  set_names(sprintf('tv=%1.2f', Bs))



In [ ]:
on_sample = function(f, mu, sigma, n) {
    X     = runif(n)
    Xtest = runif(n)
    epsilon     = sigma*rnorm(length(X))
    epsilontest = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    Ytest = mu(Xtest) + epsilontest
    dataset = list(mu=mu,X=X,Y=Y,Xtest=Xtest,Ytest=Ytest)
    f(dataset)
}

summarize_fit = function(models) {
  function(d) {
    map(models, \(fit.model) { 
      model = fit.model(d$X,d$Y)
      mu.hat = prediction.function(model)
      errors |> map(\(e) data.frame(error=e(mu.hat, d))) |>
                list_rbind(names_to='error.measure') |>
                mutate(B=model$B)
    }) |> list_rbind(names_to='model')
  }
} 

In [ ]:
tab = 1:10 |> map(\(rep) { 
  mus |> map(\(mu) 
  summarize_fit(models) |> 
  on_sample(mu, sigma, n)
  ) |> list_rbind(names_to='mu')
}) |> list_rbind(names_to='rep') 


Here's what we get.


In [ ]:
ggplot(tab, 
       aes(x=B, y=error, linetype=error.measure, shape=error.measure)) + 
  geom_point(alpha=.2, position=position_jitter(w=.1)) + 
  stat_summary(geom='line',  fun=mean) +
  stat_summary(geom='point', fun=mean) +
  facet_grid(cols=vars(mu))



### Why Test Error and Population MSE agree so well

What we see in this plot is that Test Error and $\text{Population MSE} + \sigma^2$ are very close. That means we can effective choose our variation budget $B$ to minimize Population MSE. 
That's something we can't compute in practice because we'd need knowledge of $\mu$ and the distribution of $X$ to do it, but because test error tracks it so well, we get the same effect by choosing $B$ to minimize test error.
Test Error is, in fact, an *unbiased* estimate of $\text{Population MSE} + \sigma^2$. This is a pretty straightforward calculation. Letting $\tilde E$ denote expectation conditional on the training observations ...

$$
\begin{aligned}
\tilde{E} \{ \tilde Y_i - \hat\mu(\tilde X_i)\}^2 
&= \tilde{E} \{\tilde \varepsilon_i + \mu(\tilde X_i) - \hat \mu(\tilde X_i)\}^2 \quad \text{ for } \quad \tilde \varepsilon_i = \tilde Y_i - \mu(\tilde X_i) \\ 
&= \tilde{E} \tilde\varepsilon_i^2 + \tilde{E} \{\mu(\tilde X_i) - \hat\mu(\tilde X_i)\}^2 +
  2\tilde{E} \tilde \varepsilon_i \{\mu(\tilde X_i) - \hat\mu(\tilde X_i)\} \\
&= \sigma^2 + \text{PMSE} + 0
\end{aligned}
$$

Unbiasedness is, of course, not the whole story. Some unbiased estimates are usually nowhere near the quantity they're estimating. But this isn't just an unbiased estimate. It's an *average* of $n$ independent terms, each of which is an unbiased estimate of the same quantity. 

::: {.callout-exercise}
**Optional**. Roughly how far apart, as a function of sample size $n$, would you expect them to be? Does what you say require the noise $\varepsilon_i$ to be independent of $X_i$ or is it true in greater generality?

*Hint.* What is the expectation of adjusted test error conditional on the training observations? And what is the conditional variance?
:::

::: {.callout-solution}
Let's think about the conditional variance of our test error $T = \frac{1}{n}\sum_{i=1}^n \{ \tilde Y_i - \hat\mu(\tilde X_i) \}^2$, i.e.
$\tilde{E} \{ T - \tilde{E} T \}^2$, the variance of $T$ when we think of $\hat\mu$ as a non-random function. When we think of $\hat\mu$
as nonrandom, $T$ is the mean of $n$ independent and identically distributed terms $T_i = \{\tilde Y_i - \hat\mu(\tilde X_i)\}^2$, and its variance 
is $1/n$ times the variance $\operatorname*{Var}[T_i]$ of one of them. You can calculate that if you really want to, but what you can take away from this without doing the calculation is that the standard deviation of $T$ --- the typical distance between $T$ and $\text{Population MSE} + \sigma^2$ --- is on the order of $\frac{1}{\sqrt{n}}$. 

This doesn't require independence of $\varepsilon_i$ and $X_i$. It's about the independence of $(X_i,\varepsilon_i)$ for different $i$.

:::

## Overfitting

::: {.callout-exercise}
Explain why training error tends to zero as $B$ increases. Why does test error not have this problem?
:::

# Model selection

Since the variation budget $B$ matters, it's important to choose it well. Ideally, we'd do it automatically. Give it a shot.
Propose and implement an automated approach to choosing the variation budget $B$ in the block below.

**Hint**. Do we have an error measure that is both sensible and computable? To keep things simple, I'll ask that you choose from a finite set of options $\{B_1 \ldots B_k\}$. 

In [ ]:
select.budget = function(X,Y,Bs) {
  max(Bs) # You can do better than this
}


Let's see how well this works. We'll look at error varies as a function of sample size for bounded variation regression with a few fixed budgets, as well as with a budget chosen by this procedure.
To do this, we'll make a table of errors at different sample sizes, then do some visualization. Since everything is random, we'll do all this 10 times, average the results, and take a look at how variable these results are. 

Run the block below to make the table. It may take a few minutes. Keep in mind that if you haven't finished some of the previous exercises, the results you'll get might look pretty odd.
For example, if your version of the function `bvreg` above is really just monotone regression, varying the budget won't do anything. And if you haven't implemented the model selection procedure, you'll just get a comparison of the fixed budget-approaches. 

In [ ]:
on_samples = function(f, mu, sigma, ns) {
    X     = runif(max(ns))
    Xtest = runif(max(ns))
    epsilon     = sigma*rnorm(length(X))
    epsilontest = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    Ytest = mu(Xtest) + epsilontest
    ns |> map(\(n) {
      dataset = list(mu=mu,X=X[1:n],Y=Y[1:n],Xtest=Xtest[1:n],Ytest=Ytest[1:n])
      f(dataset) |> mutate(n=n)
    }) |> list_rbind()
}
bvmodel = function(B) { function(X,Y) { bvreg(X, Y, B=B) }}
bvselected = function(Bs) {
  function(X,Y) { 
    B = select.budget(X,Y,Bs)
    bvreg(X,Y,B=B)
  }
}
models  = list(bv0.5      = bvmodel(.5),
               bv1        = bvmodel(1),
               bv2        = bvmodel(2),
               bv4        = bvmodel(4),
               bvselected = bvselected(c(.5,1,2,4)))


ns = c(25,50,100)
grid = expand_grid(rep=1:10, mu=names(mus))
tab = grid |> 
  pmap(\(rep, mu) on_samples(summarize_fit(models), mus[[mu]], sigma, ns) |> mutate(rep=rep, mu=mu)) |>
  list_rbind() 


Now run this block to plot the error curves for each of our models. What we're seeing in bold color are the means of our error measures plus/minus one standard error. This gives us a sense of which method performs best of average. To illustrate how random performance is---how much it varies when we repeat the experiment---we'll also plot the error curves for each of our 10 trials faintly. This is called a *spaghetti plot*. They tend to be a bit of a mess, but they do give you an opportunity to get a gestalt impression of what's going on without committing to any particular summary statistic.

In [ ]:
for(mu in names(mus)) {
  plot.data = tab[tab$mu == mu, ]
  print(ggplot(plot.data, aes(x=n, y=error, color=model)) + 
    geom_line(aes(group=interaction(model,rep), color=model), alpha=.3, linewidth=.5) +
    stat_summary(geom='line',  fun=mean, position=position_dodge(width = 3)) +
    stat_summary(geom='pointrange', fun.data=mean_se, position=position_dodge(width=3)) +
    facet_grid(cols=vars(error.measure)) + ggtitle(mu))
}


How does your model selection code perform relative to the other methods? Does it tend to come in first, second, etc in terms of its accuracy? Why? 